# Bonds and clusters

## Importing packages

In [ ]:
# Importing necessary packages:
from glob import glob
import numpy as np
import pandas as pd
import itertools
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

from polyphys.visualize import plotter
from polyphys.visualize.plotter import PROJECT_DETAILS as PD
from polyphys.visualize.plotter import ATTRIBUTE_LABELS as AL
from polyphys.visualize.plotter import AMIRHSI_COLORS as AC
from polyphys.visualize.plotter import DUTCH_FEILD_COLORS as DF_colors
from polyphys.visualize import tuner
from polyphys.manage.parser import \
    SumRuleCyl, TransFociCyl, TransFociCub, HnsCub
from polyphys.manage import organizer
dmon_l_c_pal = ['orange', 'darkgoldenrod', 'forestgreen']

In [ ]:
import scipy
def fit_exponential_decay(x, y):
    r"""Fit a function to an exponential decay

    .. math::  y = \exp\left(- \frac{x}{a}\right)

    Parameters
    ----------
    x, y : array_like
      The two arrays of data

    Returns
    -------
    a : float
      The coefficient *a* for this decay

    Notes
    -----
    This function assumes that data starts at 1.0 and decays to 0.0

    """
    def exp_func(x, a):
        return np.exp(-x/a)
    a = scipy.optimize.curve_fit(exp_func, x, y)[0][0]
    return a
def fit_exponential_growth(x, y):
    r"""Fit a function to an exponential decay

    .. math::  y = \exp\left(- \frac{x}{a}\right)

    Parameters
    ----------
    x, y : array_like
      The two arrays of data

    Returns
    -------
    a : float
      The coefficient *a* for this decay

    Notes
    -----
    This function assumes that data starts at 1.0 and decays to 0.0

    """
    def exp_func(x, a):
        return np.exp(x/a)
    a = scipy.optimize.curve_fit(exp_func, x, y)[0][0]
    return a

## Per geometry:

In [ ]:
project = 'HnsCub'
allInOne_db = '../../Datasets/'+project+'-allInOne/' # relative
group = 'nucleoid'
geometry = 'cubic'
topology = 'ring'
phase = 'ensAvg'
equil_db = '-'.join(
    ['allInOne', project, group, 'BondCosCorrVecMon']
)
equil_db = allInOne_db + equil_db + '.csv'
bond_corr = pd.read_csv(equil_db)


In [ ]:
phase = 'ensAvg'
equil_db = '-'.join(
    ['allInOne', project, group, 'equilProps', phase]
)
equil_db = allInOne_db + equil_db + '.csv'
chainsize_equil = pd.read_csv(equil_db)

In [ ]:
# a sorted list of unique spaces in the dataset
ensemble_longs = bond_corr['ensemble_long'].unique()
project_titles = {}
bond_corr['exp_fit'] = 0
chainsize_equil['per_length'] = 0
for ensemble_long in ensemble_longs:
    s_info = HnsCub(
        ensemble_long,
        'ensemble_long',
        geometry,
        group,
        topology,
        ispath=False
    )
    bond_corr_ens = bond_corr[bond_corr['ensemble_long']==ensemble_long]
    bond_corr_ens_idx = bond_corr_ens.index
    per_length = fit_exponential_decay(
        bond_corr_ens['bin_center'],
        bond_corr_ens['bondCosineCorrVecMon-mean']
    )
    chainsize_equil.loc[
        chainsize_equil['ensemble_long']==ensemble_long, 'per_length'] = per_length
    bond_corr.loc[bond_corr_ens_idx,'exp_fit'] = \
        bond_corr_ens.loc[:, 'bin_center'].apply(lambda x: np.exp(-x/per_length))


In [ ]:
y_prop = 'bondCosineCorrVecMon-mean'
col_attr = 'nhns'
col_order = sorted(bond_corr[col_attr].unique())
hue_attr = 'phi_c_bulk_round'
hue_order = sorted(bond_corr[hue_attr].unique())
style_attr = 'dcrowd'
style_order = sorted(bond_corr[style_attr].unique())
color_palette = 'flare'
#color_palette = 'tab10'
#color_palette = ['orange', 'darkgoldenrod', 'forestgreen']
save_to = './'
ext = 'pdf'
plot_context = 'paper'
font_scale = 2
height = 4
aspect = 1.618
ylabel_pad = 50
rc_params= {
    'axes.facecolor': 'aliceblue',
    'mathtext.default': 'regular',
    'text.usetex': True,
    'axes.grid': True,
    'axes.grid.axis': 'both',
    #'axes.grid.which': 'both'
}
facet_kws = {
    'sharey': False,
    'sharex': False,
    'legend_out': True,
}
fig_title_kws = {'fontsize': 34, 'x': 0.5, 'y': 0.93}
loc='lower left'
font_family = 'sans-serif'
move_legend_kws = {
    'ncol': 1,
    'bbox_to_anchor': (1., 0.95),
    'frameon': True,
    'facecolor': 'aliceblue',
    #'borderpad': 0.1,
    'markerscale': 1.5
}
sns.set_theme(
    context=plot_context,
    style='ticks',
    palette=color_palette,
    font='Times New Roman',
    font_scale=font_scale,
    rc=rc_params
)
bond_grid = sns.relplot(
    data=bond_corr,
    x='bin_center',
    y=y_prop,
    row=col_attr,
    row_order=col_order,
    hue=hue_attr,
    hue_order=hue_order,
    style=style_attr,
    style_order=style_order,
    kind='line',
    markers=False,
    legend='full',
    height=height,
    aspect=aspect,
    palette=color_palette,
    facet_kws=facet_kws,
    errorbar=None
)
bond_grid.set_xlabels('Bond index lag, j')
#bond_grid.set_ylabels(r'$\langle\cos(\theta_{i,i+j})\rangle$')
bond_grid.set_ylabels(r'$C(j)=\langle\hat{l}_i.\hat{l}_{i+j}\rangle$')
bond_grid.set_titles(r'$N=200, n_{{hns}}={row_name}$')
legend_labels = [attr_labels[hue_attr]] + list(hue_order)  + \
        [attr_labels[style_attr]] + list(style_order)
for idx, new_label in enumerate(legend_labels):
    bond_grid._legend.texts[idx].set_text(new_label)
#bond_grid.tight_layout(w_pad=1)
sns.move_legend(bond_grid, loc='upper left',  **move_legend_kws)
output = "-".join(["equilPlot", project, col_attr, hue_attr, 'bondCosineCorr'])
bond_grid.fig.tight_layout(pad=1)
bond_grid.savefig(save_to + output + '.' + ext, bbox_inches='tight', dpi=300)

## no crowding no bridging:

In [ ]:
dcrowd_cond = (chainsize_equil['dcrowd'].isin([1.0]))
phi_c_cond = (chainsize_equil['phi_c_bulk_round'].isin([0]))
nhns_cond = (chainsize_equil['nhns'].isin([0]))
conds = phi_c_cond & nhns_cond & dcrowd_cond
per_length = chainsize_equil.loc[conds,'per_length'].values[0]
per_length

In [ ]:
dcrowd_cond = (bond_corr['dcrowd'].isin([2.0]))
phi_c_cond = (bond_corr['phi_c_bulk_round'].isin([0]))
nhns_cond = (bond_corr['nhns'].isin([0]))
conds = phi_c_cond & nhns_cond & dcrowd_cond
data = bond_corr.loc[conds,:].copy()
data.loc[data['bin_center']>100,'exp_fit'] = data.loc[data['bin_center']<=100,'exp_fit']

hue_attr = 'dcrowd'
hue_order = sorted(bond_corr[hue_attr].unique())

color_palette = 'tab10'
#color_palette = ['orange', 'darkgoldenrod', 'forestgreen']
save_to = './'
ext = 'pdf'
plot_context = 'paper'
font_scale = 2
height = 4
aspect = 1.618
ylabel_pad = 50
rc_params= {
    #'axes.facecolor': 'aliceblue',
    'mathtext.default': 'regular',
    'text.usetex': True,
    #'axes.grid': True,
    #'axes.grid.axis': 'both',
    #'axes.grid.which': 'both'
}
facet_kws = {
    'sharey': False,
    'sharex': False,
    'legend_out': True,
}
fig_title_kws = {'fontsize': 34, 'x': 0.5, 'y': 0.93}
loc='lower left'
font_family = 'sans-serif'
move_legend_kws = {
    #'ncol': 1,
    #'bbox_to_anchor': (1., 0.95),
    'frameon': False,
    #'facecolor': 'aliceblue',
    #'borderpad': 0.1,
    'markerscale': 1
}
fig, ax = plt.subplots(nrows=1, ncols=1)#,figsize=((16/9)*3, 3))
sns.set_theme(
    context=plot_context,
    style='ticks',
    palette=color_palette,
    font='Times New Roman',
    font_scale=font_scale,
    rc=rc_params
)
bond_grid = sns.scatterplot(
    data=data,
    x='bin_center',
    y='bondCosineCorrVecMon-mean',
    legend=False,
    alpha=0.7,
    ax=ax
)
bond_grid = sns.lineplot(
    data=data,
    x='bin_center',
    y='exp_fit',
    alpha=0.7,
    lw=1,
    color='red',
    label=r'$\exp(-l_bj/l_p)$ for $j\le 100$',
    errorbar=None,
    ax=ax
)
bond_grid.set_xlabel('Bond index lag, j')
#bond_grid.set_ylabels(r'$\langle\cos(\theta_{i,i+j})\rangle$')
bond_grid.set_ylabel(r'$C(j)=\langle\hat{l}_i.\hat{l}_{i+j}\rangle$')
bond_grid.text(120, 0.7, fr'$l_p/l_b={round(per_length,3)}$')
bond_grid.set_title(r'$N=200, \phi_c=0, n_{{hns}}=0, a_c/a=2.0$')
bond_grid.legend(**move_legend_kws)
output = "-".join(["equilPlot", project,'bondCosineCorr-no_ncrowd-no_nhns'])
plt.savefig(save_to + output + '.' + ext, bbox_inches='tight', dpi=300)

In [ ]:
dcrowd_cond = (bond_corr['dcrowd'].isin([2.0]))
phi_c_cond = (bond_corr['phi_c_bulk_round'].isin([0]))
nhns_cond = (bond_corr['nhns'].isin([0]))
conds = phi_c_cond & nhns_cond & dcrowd_cond
data = bond_corr.loc[conds,:].copy()

## all persistence:

In [ ]:
dcrowd_cond = (bond_corrs['dcrowd'].isin([2.0]))
phi_c_cond = (bond_corrs['phi_c_bulk_round'].isin([0]))
nhns_cond = (bond_corrs['nhns_cond'].isin([0]))
conds = dcrowd_cond & phi_c_cond & nhns_cond
data = bond_corrs.loc[conds,:].copy()

x_prop = 'bin_center'
y_prop = 'bondCosineCorrVecMon-mean'

#row_attr = 'phi_c_bulk_round'
#row_order = norm_props

col_attr = 'phi_c_bulk_round'
col_wrap = 3
#col_order = sorted(chainsize_melted[col_attr].unique())

#style_attr = 'dcrowd'
#style_order = sorted(chainsize_melted[style_attr].unique())
#markers=['o', 'o', '^'],
#dashes=[(2, 2), (2, 2), (1, 0)],

#size_attr = 'nhns'
#size_order = sorted(chainsize_melted[size_attr].unique())
#sizes=[1+1.5*i for i in range(len(size_order))]

# The three lines below are for transc foci
hue_attr = 'nhns'
#hue_order = sorted(chainsize_melted[hue_attr].unique())
#color_palette = sns.set_palette('magma_r', len(hue_order))
#color_palette = 'rocket_r'
#color_palette = 'hls'
grid = sns.relplot(
    data=data,
    x=x_prop,
    y=y_prop,
    col=col_attr,
    col_wrap=3,
    #col_order=col_order,
    #row=row_attr,
    #row_order=row_order,
    hue=hue_attr,
    #hue_order=hue_order,
    #style=style_attr,
    #style_order=style_order,
    #size=size_attr,
    #size_order=size_order,
    #sizes=sizes,
    #markers=True,
    #markers=['s','o'],
    #dashes=False,
    legend='full',
    kind='line',
    errorbar=None,
    #height=height,
    #aspect=aspect,
    #palette=color_palette,
    #facet_kws=facet_kws,
    #markersize=15,
#    markevery=10
)


In [ ]:
color_palette = AC
#color_palette = ['orange', 'darkgoldenrod', 'forestgreen']
save_to = './'
ext = 'pdf'
plot_context = 'talk'
font_scale = 1.5
height = 5
aspect = 1.5*1.618
ylabel_pad = 50
rc_params= {
    #'axes.facecolor': 'aliceblue',
    'mathtext.default': 'regular',
    'text.usetex': True,
    #'axes.grid': True,
    #'axes.grid.axis': 'both',
    #'axes.grid.which': 'both'
}
facet_kws = {
    'sharey': 'row',
    'sharex': True,
    'legend_out': True,
}
fig_title_kws = {'fontsize': 34, 'x': 0.5, 'y': 0.93}
loc='center right'
font_family = 'sans-serif'
legend_global_kws = {
    'frameon': True,
    'facecolor': 'white',
    'framealpha': 1.0,
    #'borderpad': 0.1,
    'edgecolor': 'black',
    #'loc': 'lower left',
}
move_legend_kws = {
    'ncol': 1,
    'bbox_to_anchor': (0.8,1),
    'frameon': True,
    'facecolor': 'white',
    'framealpha': 1.0,
    #'borderpad': 0.1,
    'edgecolor': 'black',
    'markerscale': 3,
    'title': ''
}
sns.set_theme(
    context=plot_context,
    style='ticks',
    palette=color_palette,
    font='Times New Roman',
    font_scale=font_scale,
    rc=rc_params
)

In [ ]:
for ax, prop, ystep in zip(grid.axes[:,0], row_order,[0.2,0.2,0.2]):
    ax.set_ylabel(
        properties_labels[project][prop]["symbol"],
        rotation=0,
        labelpad=ylabel_pad
    )
    ax.set_title("")
#grid.set_titles(AL[row_attr] + r"$={row_name}$")
grid.set_xlabels(AL[x_prop])
grid.tight_layout(w_pad=1)
#legend_labels = [AL[hue_attr]] + list(hue_order)  + \
#        [AL[style_attr]] + list(style_order)
#for idx, new_label in enumerate(legend_labels):
#    grid._legend.texts[idx].set_text(new_label)
grid.legend.set_title(AL[hue_attr])
grid.set(xticks=np.arange(0,0.45,0.1),yticks=np.arange(0,1.1,0.2))
if style_attr == 'dcrowd':
    style_attr_name = style_attr + '-' + project
legend_labels = [AL[hue_attr]] + list(hue_order)  + \
        [AL[style_attr_name]] + list(style_order)
grid._legend.set_title('')
for idx, new_label in enumerate(legend_labels):
    grid._legend.texts[idx].set_text(new_label)
#sns.move_legend(grid, loc=loc, **move_legend_kws)
#sns.move_legend(grid, loc='upper left', **move_legend_kws)
#sns.move_legend(grid, loc=loc, **move_legend_kws)
#grid.fig.tight_layout(pad=1)
# globe size line
hns_cond = (chainsize_equil['nhns'] == 0)
phi_c_cond = (chainsize_equil['phi_c_bulk_round'] == 0)
dcrowd_cond = (chainsize_equil['dcrowd'] == 2)
gyr_phic0_hns0_ac2 = chainsize_equil.loc[hns_cond & phi_c_cond & dcrowd_cond, 'gyrMon-mean'].values[0]
nmon = chainsize_equil['nmon'].values[0]
globe_size = nmon**(1/3)
globe_line = grid.axes.flatten()[0].axhline(y=globe_size/gyr_phic0_hns0_ac2,ls=':')
grid.axes.flatten()[0].legend([globe_line],
                          [r"${R_{globe}}/{R_{g,0}}=aN^{1/3}/{R_{g,0}}$"],
                          **legend_global_kws)
# saving
output = "-".join(["equilPlot", project, x_prop, row_attr, hue_attr,style_attr, 'chainMeasures'])
grid.savefig(save_to + output + '.' + ext, bbox_inches='tight', dpi=300)
plt.close()